# Imports

In [1]:
pip install pymysql sqlalchemy-redshift pykeepass

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pykeepass import PyKeePass
import logging
import getpass
import os
from datetime import datetime
import pandas as pd
import numpy as np
import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
import boto3 #Save in S3
from pathlib import Path

In [3]:
S3_bucket_name = 'edfred-edfre-sbx-eu-west-1-solar-radiation-data'
S3_CSV_FOLD = r'EtudeWindIndex/Real'

# Accès aux données Keepass

In [4]:
#Keepass credential + logger set-up
#Get logging information stored in keepass
Configuration_path = Path('/home/ec2-user/SageMaker/EtudeWindIndex/Data/Windga/WindGa_hourly/Configuration')
KEEPASS_FILE = Configuration_path/'RDL.kdbx'

logging.basicConfig(format=' %(asctime)s -  %(levelname)s -  %(message)s', handlers = [logging.StreamHandler()])
logging.getLogger().setLevel(logging.INFO)

print('Provide password to get acces to Keepass file:')
password = getpass.getpass()

try:  
    kp = PyKeePass(KEEPASS_FILE, password=password)
    logging.info('Keepass data loaded')
except:
    logging.exception('Cannot open the keepass file:')

Provide password to get acces to Keepass file:


 ·········


 2022-01-28 10:49:53,375 -  INFO -  Keepass data loaded


In [21]:
# TEMP
kp.find_entries(title='PWU-RSH', first=True).url

'redshift+psycopg2://@rsh-eu-west-1a-exposure-warehouse-enduser.cs3rrvwot6nc.eu-west-1.redshift.amazonaws.com:5439/dwh'

# Connection à la base de données

In [22]:
def connexion_setup(kp_con_name='SBX-RDS'):

    credential = kp.find_entries(title=kp_con_name, first=True)

    con = credential.url.split('//')[0] + '//' + \
    credential.username +':'+ \
    credential.password + \
    credential.url.split('//')[1]
    logging.debug(f'connexion string: {con}')

    con_engine = sqlalchemy.create_engine(con)

    return con_engine

In [23]:
#Read mapping file
VAR_MAPPING_FILE = Configuration_path/'10min_variables_iec_to_std_names_mapping.csv'
df = pd.read_csv(VAR_MAPPING_FILE)
df.head()

#Mapping dict
iec2std = df.loc[:,['iec_attribute61400','business_description']].set_index('iec_attribute61400')['business_description'].to_dict()
std2iec = df.loc[:,['iec_attribute61400','business_description']].set_index('business_description')['iec_attribute61400'].to_dict()

#Récupération des données utiles
mask_features = [False]
List_features = ['active_power_avg','wind_speed_avg']
for feature in List_features :
    mask_features = mask_features | (df.business_description == feature)
temperature_features = df.loc[mask_features,:]
features_selection_str = str(list(temperature_features.iec_attribute61400)).replace("'", '').replace("[", "").replace("]", "")

#Visualisation
features_selection_str

'wmet4_horwdspd_mag_f, wtur4_w_mag_f'

In [24]:
#Paramètres
PROJECT =  'AUQB'
MONTH = '2021-10'    # string: 'YYYY-MM'

# Extraction de données

In [45]:
#Number of WTGS in this project from WDM
query = f'''SELECT project_code, iec_eqpt_code, wt_neighbor_01, power_curve_code
            FROM wdm.adm_eqpt_wt 
            WHERE project_code = '{PROJECT}' ;
            '''
            
with connexion_setup('SBX-WDM').connect() as conn:
     wtg_eqpt_codes = pd.read_sql_query(sql=query,
                                        con=conn, 
                                        params={})
WTGS = tuple(wtg_eqpt_codes.iec_eqpt_code)

#Visualisation
WTGS

('AUQB-ECP001-TUR006',
 'AUQB-ECP001-TUR007',
 'AUQB-ECP001-TUR008',
 'AUQB-ECP001-TUR009',
 'AUQB-ECP001-TUR010')

In [56]:
#10 minutes data extraction
FROM_DATE =  MONTH + '-01 00:00:00'  # included
if int(MONTH[-2:])==12:
    TO_DATE = f'{int(MONTH[:-3]) + 1}-01-01 00:00:00'      # not included 
elif int(MONTH[-2:])>=9:
    TO_DATE = f'{MONTH[:-3]}-{int(MONTH[-2:]) + 1}-01 00:00:00'      # not included 
else :
    TO_DATE = f'{MONTH[:-3]}-0{int(MONTH[-2:]) + 1}-01 00:00:00'      # not included 
    TO_DATE = f'2021-12-01 00:00:00'      # not included


DATES = [d.strftime('%Y%m%d') for d in pd.date_range('2020-01', '2021-11')]
_10min_tables = [f'eu_data.tur_10m_{date}_q' for date in DATES[:-1]]
query = ""

for i, table in enumerate(_10min_tables):
    if i != 0:
        query = query + "UNION\n"
    query = query + f'''SELECT  asset_id,
                                ts,
                                {features_selection_str}
                        FROM {table} WHERE project = '{PROJECT}' AND tech_source = 'PIOEM' \n'''

query = query + "ORDER BY ts ;"

with (connexion_setup('PWU-RSH').connect()) as conn:
    _10min = pd.read_sql_query(sql=query, con=conn)    # query_red(conn, query)
  
_10min.rename(columns=iec2std,
             inplace=True)

#_10min.set_index(['asset_id', 'ts'], inplace=True)

# re-order columns
col_selection = _10min.columns.sort_values()
_10min = _10min.loc[:,col_selection]
_10min.sample(6)

,active_power_avg,asset_id,ts,wind_speed_avg
400378,NaN,AUQB-ECP001-TUR009,2021-07-10 01:50:00,NaN
195717,1193.0,AUQB-ECP001-TUR009,2020-09-28 19:50:00,10.7
224306,1677.0,AUQB-ECP001-TUR008,2020-11-07 12:50:00,13.2
85031,0.0,AUQB-ECP001-TUR006,2020-04-28 02:20:00,6.3
214573,-1.0,AUQB-ECP001-TUR010,2020-10-25 00:20:00,3.9
185214,246.0,AUQB-ECP001-TUR008,2020-09-14 05:40:00,6.9


# Traitement des données

In [64]:
#On converti le timestamp
_10min['year'] = _10min.ts.map(lambda date: date.year)
_10min['month'] = _10min.ts.map(lambda date: date.month)
_10min['day'] = _10min.ts.map(lambda date: date.day)
_10min['hour'] = _10min.ts.map(lambda date: date.hour)
_10min['minute'] = _10min.ts.map(lambda date: date.minute)

#On récupère le nom de projet et le numéro de tubine
_10min['project'] = _10min.asset_id.map(lambda name: name[0:4])
_10min['turbine'] = _10min.asset_id.map(lambda name: int(name[-3:]))

AttributeError: 'DataFrame' object has no attribute 'ts'

In [58]:
#On garde les informations utiles
_10min.drop(['asset_id','ts'], axis=1, inplace=True)
#On modifie l'index pour retrouver plus facilement les données
#_10min.set_index(['project','turbine','year','month','day','hour','minute'], inplace=True)

In [59]:
#Visualisation
_10min.sample(5)

,active_power_avg,wind_speed_avg,year,month,day,hour,minute,project,turbine
180880,11.0,5.9,2020,9,8,5,20,AUQB,10
22691,NaN,NaN,2020,2,1,12,20,AUQB,8
66650,NaN,NaN,2020,4,2,13,40,AUQB,8
221376,795.0,11.4,2020,11,3,11,10,AUQB,10
66039,89.0,4.9,2020,4,1,17,10,AUQB,6


In [70]:
df= pd.DataFrame(columns=['year','month','active_power','wind_speed_data'])
for year in [2020] :
    for month in range(1,13) :
        active_power = _10min[(_10min['year']==year)&(_10min['month']==month)].active_power_avg.mean()
        wind_speed_data = _10min[(_10min['year']==year)&(_10min['month']==month)].wind_speed_avg.mean()

        df = df.append({'year':int(year),'month':int(month),'active_power':active_power,'wind_speed_data':wind_speed_data}, ignore_index=True)

for year in [2021] :
    for month in range(1,12) :
        active_power = _10min[(_10min['year']==year)&(_10min['month']==month)].active_power_avg.mean()
        wind_speed_data = _10min[(_10min['year']==year)&(_10min['month']==month)].wind_speed_avg.mean()

        df = df.append({'year':int(year),'month':int(month),'active_power':active_power,'wind_speed_data':wind_speed_data}, ignore_index=True)


In [71]:
df.to_csv()

,year,month,active_power,wind_speed_data
0,2020.0,1.0,595.947538,7.208634
1,2020.0,2.0,561.762045,7.379006
2,2020.0,3.0,478.139648,6.764127
3,2020.0,4.0,174.675534,4.843311
4,2020.0,5.0,299.265257,6.028793
5,2020.0,6.0,587.058708,7.794566
6,2020.0,7.0,432.150937,6.973362
7,2020.0,8.0,416.076033,6.777205
8,2020.0,9.0,339.604676,6.021138
9,2020.0,10.0,581.486196,7.681961


# Extraction globale

In [33]:
#Liste des projets à extraire
start_project = 'AUQB'
start_month = '2021-01'
end_month = '2021-11'

In [34]:
months_range = pd.date_range(start=start_month, end=end_month, freq='MS')
months_list = [str(months_range[i])[0:7] for i in range(len(months_range))]

#On récupère le lsite des projets
ERA5_monthly_path = Path('/home/ec2-user/SageMaker/EtudeWindIndex/Data/ERA5/ERA5_monthly/Clean/ERA5_monthly.csv')
era5 = pd.read_csv(ERA5_monthly_path, sep=';')
List_projects = era5.project_code.unique().tolist()[era5.project_code.unique().tolist().index(start_project):]
List_projects = ['AUQB']

In [35]:
for PROJECT in List_projects :
    
    logging.info("project : {}".format(PROJECT))
    
    _10min_project = pd.DataFrame(columns=['asset_id','ts'] + List_features)
    
    #Number of WTGS in this project from WDM
    query = f'''SELECT project_code, iec_eqpt_code, wt_neighbor_01, power_curve_code
    FROM wdm.adm_eqpt_wt 
    WHERE project_code = '{PROJECT}' ;
    '''
    
    with connexion_setup('SBX-WDM').connect() as conn:
        wtg_eqpt_codes = pd.read_sql_query(sql=query, con=conn, params={})
    
    WTGS = tuple(wtg_eqpt_codes.iec_eqpt_code)
        
    #10 minutes data extraction
    for MONTH in months_list :
        FROM_DATE =  MONTH + '-01 00:00:00'  # included
        if int(MONTH[-2:])==12:
            TO_DATE = f'{int(MONTH[:-3]) + 1}-01-01 00:00:00'      # not included 
        elif int(MONTH[-2:])>=9:
                TO_DATE = f'{MONTH[:-3]}-{int(MONTH[-2:]) + 1}-01 00:00:00'      # not included 
        else :
            TO_DATE = f'{MONTH[:-3]}-0{int(MONTH[-2:]) + 1}-01 00:00:00'      # not included 
            TO_DATE = f'2021-12-01 00:00:00'      # not included
            
        DATES = [d.strftime('%Y%m%d') for d in pd.date_range(FROM_DATE, TO_DATE)]
        _10min_tables = [f'eu_data.tur_10m_{date}_q' for date in DATES[:-1]]
        query = ""
        
        for i, table in enumerate(_10min_tables):
            if i != 0:
                query = query + "UNION\n"
            query = query + f'''SELECT  asset_id, 
            ts,
            {features_selection_str}
            FROM {table} WHERE project = '{PROJECT}' AND tech_source = 'PIOEM' \n'''
            
        query = query + "ORDER BY ts ;"
        
        with (connexion_setup('PWU-RSH').connect()) as conn:
            _10min = pd.read_sql_query(sql=query, con=conn)    # query_red(conn, query)
            
        _10min.rename(columns=iec2std, inplace=True)
        
        # re-order columns
        col_selection = _10min.columns.sort_values()
        _10min = _10min.loc[:,col_selection]
        
        _10min_project = pd.concat([_10min_project, _10min])
        
    #On converti le timestamp
    _10min_project['year'] = _10min_project.ts.map(lambda date: date.year)
    _10min_project['month'] = _10min_project.ts.map(lambda date: date.month)
    _10min_project['day'] = _10min_project.ts.map(lambda date: date.day)
    _10min_project['hour'] = _10min_project.ts.map(lambda date: date.hour)
    _10min_project['minute'] = _10min_project.ts.map(lambda date: date.minute)
    
    #On récupère le nom de projet et le numéro de tubine
    _10min_project['project'] = _10min_project.asset_id.map(lambda name: name[0:4])
    _10min_project['turbine'] = _10min_project.asset_id.map(lambda name: int(name[-3:]))
            
    for year in [2020] :
        for month in [1,2,3,4,5,6,7,8,9,10,11,12] :
            _10min_project[(_10min_project['year']==year)&(_10min_project['month']==month)].active_power_avg
            _10min_project[(_10min_project['year']==year)&(_10min_project['month']==month)].wind_speed_avg
            
    for year in [2021] :
        for month in [1,2,3,4,5,6,7,8,9,10,11] :
            _10min_project[(_10min_project['year']==year)&(_10min_project['month']==month)].active_power_avg
            _10min_project[(_10min_project['year']==year)&(_10min_project['month']==month)].wind_speed_avg
    
    #On garde les informations utiles
    _10min_project.drop(['asset_id','ts'], axis=1, inplace=True)
    #On modifie l'index pour retrouver plus facilement les données
    _10min_project.set_index(['project','turbine','year','month','day','hour','minute'], inplace=True)

    #Sauvegarde sur le notebook
    _10min_path = Path('/home/ec2-user/SageMaker/EtudeWindIndex/Data/Windga/WindGa_hourly/Clean/10min_'+PROJECT+'.csv')
    _10min_project.to_csv(_10min_path, index=False, sep=';')
    
    #Sauvegarde sur le S3
    #outfile = '10min_'+PROJECT+'.csv'
    #_10min_project.to_csv(f's3://{S3_bucket_name}/{S3_CSV_FOLD}/{outfile}', index=True, sep=';')

 2022-01-28 10:56:37,387 -  INFO -  project : AUQB
